In [0]:
#DOWNLOAD & EXTRACT FILE
import zipfile
import requests
from pyspark.sql.types import *
import random
from pyspark.sql.functions import lit

url='https://www.briandunning.com/sample-data/us-500.zip'
req = requests.get(url)

filename = url.split('/')[-1]
 
# Writing the file to the local file system
with open(filename,'wb') as output_file:
    output_file.write(req.content)
print('Downloading Completed')


with zipfile.ZipFile("/databricks/driver/us-500.zip", 'r') as zip_ref:
    zip_ref.extractall("/databricks/driver")

In [0]:
#source dataframe and list of 50000 numbers
sourceDf=spark.read.format("csv").option("header",True).load("file:/databricks/driver/us-500.csv")
# sourceDf.show(10,False)
sourceDf.registerTempTable("test")

l1=[i for i in range(1,50001)]
fullSeqDf=spark.createDataFrame(l1, IntegerType())
fullSeqDf.registerTempTable("seq")
fullSeqDf.show()

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
|   11|
|   12|
|   13|
|   14|
|   15|
|   16|
|   17|
|   18|
|   19|
|   20|
+-----+
only showing top 20 rows



In [0]:
#dataset prepare
names=spark.sql("""
with names as (
  SELECT first_name as name FROM test
  ),
names_with_nums as (
  select row_number() OVER (ORDER BY n1.name,n2.name) as id,
    n1.name as first_name,
    n2.name as last_name,
    lower(n1.name)||'.'||lower(n2.name)||"@gmail.com" as email_id
  FROM names as n1
  CROSS JOIN names as n2
  WHERE n1.name!=n2.name)
select * FROM names_with_nums
WHERE id<=50001
""")

names.registerTempTable("names")
# names.show()

locations=spark.sql("""
WITH loc_prepare AS (
SELECT DISTINCT city,county,state,zip FROM test )
SELECT (row_number() over (order by city,county,state,zip))-1 loc_id,*
FROM loc_prepare
""")

locations.registerTempTable("locations")
# locations.show()

companies=spark.sql("""
WITH com_prepare AS (
SELECT DISTINCT company_name,web FROM test )
SELECT (row_number() over (order by company_name,web))-1 loc_id,*
FROM com_prepare
""")

companies.registerTempTable("companies")
# companies.show()

addresses=spark.sql("""
with names as (
  SELECT last_name as name FROM test
  ),
names_with_nums as (
  select row_number() OVER (ORDER BY n1.name,n2.name) as id,
    n1.name as first_name,
    n2.name as last_name
  FROM names as n1
  CROSS JOIN names as n2
  WHERE n1.name!=n2.name
  ),
addresses as (
  select id,
  id||" "||first_name||" St. "|| last_name  as address FROM names_with_nums
  WHERE id<=50001
  )
SELECT * FROM addresses
""")

addresses.registerTempTable("addresses")
addresses.show()

def formPh(ph):
  strPh=str(ph)
  return f"{strPh[:3]}-{strPh[3:6]}-{strPh[6:10]}"

ph1=[formPh(9842268754+i) for i in range(0,50001)]
phone1=spark.createDataFrame(ph1, StringType())
phone1.registerTempTable("ph1")
phoneNum1=spark.sql("""SELECT row_number() OVER (ORDER BY value) as id,value no FROM ph1""")
phoneNum1.registerTempTable("phoneNum1")

ph2=[formPh(8842468354+i) for i in range(0,50001)]
phone2=spark.createDataFrame(ph2, StringType())
phone2.registerTempTable("ph2")
phoneNum2=spark.sql("""SELECT row_number() OVER (ORDER BY value) as id,value no FROM ph2""")
phoneNum2.registerTempTable("phoneNum2")

+---+--------------------+
| id|             address|
+---+--------------------+
|  1| 1 Abdallah St. Acey|
|  2|2 Abdallah St. Acuff|
|  3|3 Abdallah St. Adkin|
|  4|4 Abdallah St. Ag...|
|  5| 5 Abdallah St. Ahle|
|  6|6 Abdallah St. Al...|
|  7|7 Abdallah St. Am...|
|  8|8 Abdallah St. Amyot|
|  9|9 Abdallah St. An...|
| 10|10 Abdallah St. A...|
| 11|11 Abdallah St. A...|
| 12|12 Abdallah St. A...|
| 13|13 Abdallah St. A...|
| 14|14 Abdallah St. A...|
| 15|15 Abdallah St. A...|
| 16|16 Abdallah St. Asar|
| 17|17 Abdallah St. A...|
| 18|18 Abdallah St. A...|
| 19|19 Abdallah St. B...|
| 20|20 Abdallah St. B...|
+---+--------------------+
only showing top 20 rows



In [0]:
#form dateset

employees=spark.sql("""

with emp as (
  SELECT row_number() over (order by first_name,last_name) as empId,* FROM test
),
seqn as (
  SELECT value,int(RAND()*10000) r_val
  FROM seq 
),
emp_blow as (
  SELECT 
    s.value as emp_id,
    coalesce(t.first_name,n.first_name) as first_name,
    coalesce(t.last_name,n.last_name) as last_name,
    coalesce(t.company_name,c.company_name) as company_name,
    coalesce(t.address,a.address) as address,
    coalesce(t.city,l.city) as city,
    coalesce(t.county,l.county) as county,
    coalesce(t.state,l.state) as state,
    coalesce(t.zip,l.zip) as zip,
    coalesce(t.phone1,p1.no) as phone1,
    coalesce(t.phone2,p2.no) as phone2,
    coalesce(t.email,n.email_id) as email,
    coalesce(t.web,c.web) as web
  FROM seqn s
  LEFT JOIN emp t ON s.value=t.empId
  LEFT JOIN names n ON s.value=n.id
  LEFT JOIN locations l ON mod(s.value*s.r_val,451)=l.loc_id
  LEFT JOIN companies c ON mod(s.value*s.r_val,499)=c.loc_id
  LEFT JOIN addresses a ON s.value=a.id
  LEFT JOIN phoneNum1 p1 ON s.value=p1.id
  LEFT JOIN phoneNum2 p2 ON s.value=p2.id)
select * from emp_blow""")

employees.registerTempTable("employees")

In [0]:
%sql

SELECT * FROM employees

emp_id,first_name,last_name,company_name,address,city,county,state,zip,phone1,phone2,email,web
148,Ernie,Stenseth,Knwz Newsradio,45 E Liberty St,Ridgefield Park,Bergen,NJ,07660,201-709-6245,201-387-9093,ernie_stenseth@aol.com,http://www.knwznewsradio.com
463,Tonette,Wenner,Northwest Publishing,4545 Courthouse Rd,Westbury,Nassau,NY,11590,516-968-6051,516-333-4861,twenner@aol.com,http://www.northwestpublishing.com
471,Valentin,Klimek,"Schmid, Gayanne K Esq",137 Pioneer Way,Chicago,Cook,IL,60604,312-303-5453,312-512-2338,vklimek@klimek.org,http://www.schmidgayannekesq.com
496,Yolando,Luczki,Dal Tile Corporation,422 E 21st St,Syracuse,Onondaga,NY,13214,315-304-4759,315-640-6357,yolando@cox.net,http://www.daltilecorporation.com
833,Adelina,Maryann,"Chemel, James L Cpa",833 Acey St. Pagliuca,Miami,Miami-Dade,FL,33196,984-226-9586,884-246-9186,adelina.maryann@gmail.com,http://www.chemeljameslcpa.com
1088,Adell,Christiane,K Cs Cstm Mouldings Windows,1088 Acuff St. Coyier,Nutley,Essex,NJ,07110,984-226-9841,884-246-9441,adell.christiane@gmail.com,http://www.kcscstmmouldingswindows.com
1238,Adell,Jovita,"Arturi, Peter D Esq",1238 Acuff St. Kulzer,South San Francisco,San Mateo,CA,94080,984-226-9991,884-246-9591,adell.jovita@gmail.com,http://www.arturipeterdesq.com
1342,Adell,Micaela,Nercon Engineering & Mfg Inc,1342 Acuff St. Pawlowicz,San Ramon,Contra Costa,CA,94583,984-227-0095,884-246-9695,adell.micaela@gmail.com,http://www.nerconengineeringmfginc.com
1580,Ahmed,Chantell,Glenridge Manor Mobile Home Pk,1580 Adkin St. Coody,Newport News,Newport News City,VA,23608,984-227-0333,884-246-9933,ahmed.chantell@gmail.com,http://www.glenridgemanormobilehomepk.com
1591,Ahmed,Clorinda,"Potter, Brenda J Cpa",1591 Adkin St. Cryer,Seattle,King,WA,98104,984-227-0344,884-246-9944,ahmed.clorinda@gmail.com,http://www.potterbrendajcpa.com


In [0]:
#Create a dataframe of CityEmployeeDensity, the 1st city will be the one with maxium number of employees

CityEmployeeDensity=spark.sql(""" WITH emp_counts as (
   SELECT city,COUNT(*) as emp_count
   FROM employees
   GROUP BY city
 )
 SELECT city,emp_count,row_number() over (order by emp_count desc) as vaccination_order
 FROM emp_counts""")

CityEmployeeDensity.show(500,False)
CityEmployeeDensity.registerTempTable("CityEmployeeDensity")


  


+-------------------+---------+-----------------+
|city               |emp_count|vaccination_order|
+-------------------+---------+-----------------+
|New York           |1077     |1                |
|Philadelphia       |798      |2                |
|Chicago            |769      |3                |
|Miami              |685      |4                |
|Milwaukee          |600      |5                |
|Orlando            |595      |6                |
|Los Angeles        |540      |7                |
|Baltimore          |505      |8                |
|Denver             |498      |9                |
|Providence         |493      |10               |
|Indianapolis       |491      |11               |
|Atlanta            |416      |12               |
|Brooklyn           |414      |13               |
|Phoenix            |406      |14               |
|Aberdeen           |395      |15               |
|Cincinnati         |325      |16               |
|San Jose           |322      |17               |


In [0]:
VaccinationDrivePlan=employees.withColumn("Sequence",lit(-1))
VaccinationDrivePlan.show()
VaccinationDrivePlan.registerTempTable("VaccinationDrivePlan")

+------+----------+----------+--------------------+--------------------+-------------------+-----------------+-----+-----+------------+------------+--------------------+--------------------+--------+
|emp_id|first_name| last_name|        company_name|             address|               city|           county|state|  zip|      phone1|      phone2|               email|                 web|Sequence|
+------+----------+----------+--------------------+--------------------+-------------------+-----------------+-----+-----+------------+------------+--------------------+--------------------+--------+
|   148|     Ernie|  Stenseth|      Knwz Newsradio|     45 E Liberty St|    Ridgefield Park|           Bergen|   NJ|07660|201-709-6245|201-387-9093|ernie_stenseth@ao...|http://www.knwzne...|      -1|
|   463|   Tonette|    Wenner|Northwest Publishing|  4545 Courthouse Rd|           Westbury|           Nassau|   NY|11590|516-968-6051|516-333-4861|     twenner@aol.com|http://www.northw...|      -1|


In [0]:
VaccinationSequence=VaccinationDrivePlan.join(CityEmployeeDensity,CityEmployeeDensity.city==VaccinationDrivePlan.city,"left")\
                           .select(VaccinationDrivePlan.emp_id,
                                  VaccinationDrivePlan.first_name,
                                  VaccinationDrivePlan.last_name,
                                  VaccinationDrivePlan.company_name,
                                  VaccinationDrivePlan.address,
                                  VaccinationDrivePlan.city,
                                  VaccinationDrivePlan.county,
                                  VaccinationDrivePlan.state,
                                  VaccinationDrivePlan.zip,
                                  VaccinationDrivePlan.phone1,
                                  VaccinationDrivePlan.phone2,
                                  VaccinationDrivePlan.email,
                                  VaccinationDrivePlan.web,
                                  CityEmployeeDensity.vaccination_order.alias('sequence'))
VaccinationSequence.show()

VaccinationSequence.registerTempTable("VaccinationSequence")



+------+----------+----------+--------------------+--------------------+-------------------+-----------------+-----+-----+------------+------------+--------------------+--------------------+--------+
|emp_id|first_name| last_name|        company_name|             address|               city|           county|state|  zip|      phone1|      phone2|               email|                 web|sequence|
+------+----------+----------+--------------------+--------------------+-------------------+-----------------+-----+-----+------------+------------+--------------------+--------------------+--------+
|   148|     Ernie|  Stenseth|      Knwz Newsradio|     45 E Liberty St|    Ridgefield Park|           Bergen|   NJ|07660|201-709-6245|201-387-9093|ernie_stenseth@ao...|http://www.knwzne...|     269|
|   463|   Tonette|    Wenner|Northwest Publishing|  4545 Courthouse Rd|           Westbury|           Nassau|   NY|11590|516-968-6051|516-333-4861|     twenner@aol.com|http://www.northw...|     182|


In [0]:
VaccinationSchedule=spark.sql("""with vacc_order as (
                                    SELECT emp_id,city,row_number() over (partition by city order by emp_id) as position
                                    FROM VaccinationSequence
                                )
                                SELECT emp_id,city,floor(((position-1)/100))+1 as day
                                FROM vacc_order
                                order by 2,3,1
                                """)

VaccinationSchedule.show()
VaccinationSchedule.registerTempTable("VaccinationSchedule")

+------+--------+---+
|emp_id|    city|day|
+------+--------+---+
|   119|Aberdeen|  1|
|   561|Aberdeen|  1|
|   685|Aberdeen|  1|
|   861|Aberdeen|  1|
|   902|Aberdeen|  1|
|   990|Aberdeen|  1|
|  1148|Aberdeen|  1|
|  1183|Aberdeen|  1|
|  1312|Aberdeen|  1|
|  1353|Aberdeen|  1|
|  1568|Aberdeen|  1|
|  1681|Aberdeen|  1|
|  1710|Aberdeen|  1|
|  1804|Aberdeen|  1|
|  2058|Aberdeen|  1|
|  2079|Aberdeen|  1|
|  2211|Aberdeen|  1|
|  2222|Aberdeen|  1|
|  2255|Aberdeen|  1|
|  2412|Aberdeen|  1|
+------+--------+---+
only showing top 20 rows



In [0]:
VaccinationScheduleSeq=spark.sql("""
                                    with vacc_order as (
                                    SELECT emp_id,city,row_number() over (order by sequence,emp_id) as position
                                    FROM VaccinationSequence
                                )
                                SELECT emp_id,city,floor(((position-1)/100))+1 as day
                                FROM vacc_order
                                order by 3,2,1
                                 """)
VaccinationScheduleSeq.show()
VaccinationScheduleSeq.registerTempTable("VaccinationScheduleSeq")

+------+--------+---+
|emp_id|    city|day|
+------+--------+---+
|    13|New York|  1|
|    58|New York|  1|
|   103|New York|  1|
|   118|New York|  1|
|   157|New York|  1|
|   194|New York|  1|
|   225|New York|  1|
|   237|New York|  1|
|   246|New York|  1|
|   287|New York|  1|
|   348|New York|  1|
|   373|New York|  1|
|   451|New York|  1|
|   490|New York|  1|
|   606|New York|  1|
|   623|New York|  1|
|   676|New York|  1|
|   739|New York|  1|
|   777|New York|  1|
|   836|New York|  1|
+------+--------+---+
only showing top 20 rows



In [0]:
estimatedCompletion=spark.sql("""SELECT city,MAX(day) as Reqd_days
FROM VaccinationSchedule
GROUP BY city
ORDER BY 2 DESC""")


estimatedCompletion.show()
estimatedCompletion.registerTempTable("estimatedCompletion")



+------------+---------+
|        city|Reqd_days|
+------------+---------+
|    New York|       11|
|Philadelphia|        8|
|     Chicago|        8|
|       Miami|        7|
|   Baltimore|        6|
| Los Angeles|        6|
|   Milwaukee|        6|
|     Orlando|        6|
|     Atlanta|        5|
|     Phoenix|        5|
|Indianapolis|        5|
|    Brooklyn|        5|
|  Providence|        5|
|      Denver|        5|
|    Aberdeen|        4|
|   Anchorage|        4|
|  Bridgeport|        4|
| New Orleans|        4|
|  Cincinnati|        4|
|    Riverton|        4|
+------------+---------+
only showing top 20 rows



In [0]:

estimatedCompletionSeq=spark.sql("""SELECT city,COUNT(DISTINCT day) as Reqd_days
FROM VaccinationScheduleSeq
GROUP BY city
ORDER BY 2 DESC""")


estimatedCompletionSeq.show()
estimatedCompletionSeq.registerTempTable("estimatedCompletionSeq") 
 
 


+------------+---------+
|        city|Reqd_days|
+------------+---------+
|    New York|       11|
|     Chicago|        9|
|Philadelphia|        9|
|       Miami|        8|
|   Milwaukee|        7|
|     Orlando|        7|
| Los Angeles|        6|
|   Baltimore|        6|
|Indianapolis|        6|
|      Denver|        6|
|  Providence|        6|
|    Brooklyn|        5|
|     Phoenix|        5|
|     Atlanta|        5|
|    Aberdeen|        5|
|  Cincinnati|        5|
|    San Jose|        4|
|  Bridgeport|        4|
|   Anchorage|        4|
|     Gardena|        4|
+------------+---------+
only showing top 20 rows



In [0]:
%sql

SELECT city,day,COUNT(*) FROM VaccinationScheduleSeq
--WHERE day between 400 and 410
GROUP BY city,day
order by 1,2

city,day,count(1)
Aberdeen,83,13
Aberdeen,84,100
Aberdeen,85,100
Aberdeen,86,100
Aberdeen,87,82
Abilene,124,43
Abilene,125,100
Abilene,126,100
Abilene,127,49
Absecon,332,32
